In [ ]:
import cv2
import numpy as np
import pywt
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def extract_frames(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (128, 128))
        frames.append(resized)
        count += 1
    cap.release()
    return frames

def apply_wavelet_transform(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    features = []
    for level in coeffs:
        if isinstance(level, tuple):
            for sub_band in level:
                features.append(np.mean(sub_band))
                features.append(np.std(sub_band))
        else:
            features.append(np.mean(level))
            features.append(np.std(level))
    return np.array(features)

def load_dataset_wavelet(video_paths, labels, max_frames=50):
    features, new_labels = [], []
    for i, path in enumerate(video_paths):
        if not os.path.exists(path):
            continue
        frames = extract_frames(path, max_frames)
        if len(frames) == 0:
            continue
        for frame in frames:
            feat = apply_wavelet_transform(frame)
            features.append(feat)
            new_labels.append(labels[i])
    return np.array(features), np.array(new_labels)

def train_decision_tree(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)
    clf = DecisionTreeClassifier(max_depth=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with Wavelet Transform (Decision Tree): {accuracy:.4f}")
    return clf

def prepare_ffpp_dataset(base_path):
    video_paths, labels = [], []
    for label, folder in enumerate(['real', 'fake']):
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            continue
        for file in os.listdir(folder_path):
            if file.endswith('.mp4'):
                video_paths.append(os.path.join(folder_path, file))
                labels.append(label)
    return video_paths, labels

base_path = "C:\\Users\\Pavan\\Downloads\\FF++"
video_paths, labels = prepare_ffpp_dataset(base_path)

try:
    features, labels = load_dataset_wavelet(video_paths, labels)
    if len(features) > 0:
        model = train_decision_tree(features, labels)
    else:
        print("No features extracted.")
except Exception as e:
    print(f"Error: {e}")
